In [15]:
# coding: utf-8
import re
import pandas as pd
import shutil
import os
import subprocess

In [16]:
#####please add the language list
lan_list = ['ENU']
#####

In [17]:
with open('c:\_workspace\cloud\\bat_files\Audi_CVT_Azure.txt', 'r') as b:
    lines =  b.readlines()
b.close()
bat_lines = []
for line in lines:
    for l in lan_list:
        if l in line:
            #print(line)
            bat_lines.append(line)
bat_file = 'Audi_CVT_Azure_lan.bat'
o = open(bat_file, 'w+')
o.writelines(bat_lines)
o.close()            
#C:\_projects\mib3-audi\dev\_tools\SPOT\spot.exe convert2wav -i "c:\_projects\mib2plus-connected\dev\tts_regression\ITI_Alice-ML\baseline\Audi_Azure_CVT"

In [18]:
cmd = 'Audi_CVT_Azure_lan.bat>output_ju.txt'
print('running regression test locally...')
os.system(cmd)
print('local regression test done')

running regression test locally...
done


## run the regression test locally with SPOT?

In [20]:
with open('output_ju.txt', 'r') as log_file_fh:
    lines = [line.strip('\n') for line in log_file_fh.readlines()]
log_file_fh.close()
work_path = 'c:\_projects\\mib2plus-connected\\dev\\tts_regression'

lan=[]
input_text = []
file_name=[]
mask_name = []
voice=[]
tar_pcm = []
ref_pcm = []
ref=[]
tar=[]
pcm = []
ref_folder = []
tar_folder=[]
pcm_folder= []

In [21]:
for i in range(len(lines)):
    if lines[i][:8] == 'FAILURE:':
        lan.append(lines[i-1].split(' ')[0])
        file_name.append(lines[i-1].split(' ')[2])
        mask_name.append(lines[i-1].split(' ')[1])
        #pcm.append(lines[i+1].split('\\')[-1])
        voice.append((lines[i+1].split('\\..\\..\\')[1]).split('\\')[1])
        ref.append((lines[i+1].split('\\..\\..\\')[1]).split('\\')[-2])
        tar.append(lines[i+2].split('\\..\\..\\')[1].split('\\')[-2])
        i+=3
print((lan[1]),(file_name[1]), mask_name[1],len(voice),ref[1],tar[1])
    

('ENU', 'input_weather_enu_00002', 'weather', 195, 'Audi_MTLDEV_Ext', 'Audi_Azure_CVT')


In [23]:
def wav_link(voice,ref,tar,f):
    ref_f =  work_path+ '\\'+voice + '\\baseline'+'\\'+ref
    tar_f = work_path+ '\\'+voice + '\\baseline'+'\\'+tar
    if ref_f not in ref_folder:
        ref_folder.append(ref_f)
    if tar_f not in tar_folder:
        tar_folder.append(tar_f)
    ref_wav = work_path+ '\\'+voice + '\\baseline'+'\\'+ref+'\\'+ f + '_ref.wav'
    tar_wav = work_path+ '\\'+voice + '\\baseline'+'\\'+tar+'\\'+ f + '.wav'
    
    #wav = work_path  +'\\'+ pcm
    ref_pcm.append(ref_wav)
    tar_pcm.append(tar_wav)
    return ref_pcm,tar_pcm,ref_folder,tar_folder

#print(wav_link(voice[1],ref[1],tar[1],file_name[1]))

In [24]:
#run the wav_link function
for i in range(len(lan)):
    wav_link(voice[i],ref[i],tar[i],file_name[i])
    
#get the link to the pcm
ref_link = ['=HYPERLINK("'+  r+ '";"open")' for r in ref_pcm]
tar_link= ['=HYPERLINK("'+  t+ '";"open")' for t in tar_pcm]

In [25]:
#creat bat to convert pcm to wav
to_wav_cmd=[]
conv2wav= 'C:\_projects\mib3-audi\dev\_tools\SPOT\spot.exe convert2wav -i' 
for n in (ref_folder+tar_folder):
    cmd = conv2wav+'\"' +n +'\"'+'\npause'
#run the pcm2wav batch file:
    print('converting pcm to wav...')
    os.system(cmd)
    print(n + ' pcm to wav converted. \n')
    to_wav_cmd.append(cmd)

#write the cmd lines to a batch file
'''
c = open('pcm_2_wav.bat', 'w+')
c.writelines(to_wav_cmd)
c.close()     
'''

c:\_projects\mib2plus-connected\dev\tts_regression\ENU_Ava-ML\baseline\Audi_MTLDEV_Ext pcm to wav converted. 

c:\_projects\mib2plus-connected\dev\tts_regression\ENU_Ava-ML\baseline\Audi_Azure_CVT pcm to wav converted. 



"\nc = open('pcm_2_wav.bat', 'w+')\nc.writelines(to_wav_cmd)\nc.close()     \n"

In [26]:
df=pd.DataFrame({'language':lan,'file':file_name, 'mask':mask_name,'ref':ref_link,'tar':tar_link})
#print(df)

In [27]:
#save the csv file
df.to_csv('cloud_rtest.csv',sep=';' ,index=False,encoding='utf-8_sig')

In [29]:
r=open('new_ref_list.txt','r')
lines = r.readlines()
r.close()
print(lines)

['Izzy\n', 'enutravelregression9\n', 'enuweatherregression281\n', 'enuweatherregression282\n', 'enuweatherdict01']


In [34]:
tar_path = 'c:\\_projects\\mib2plus-connected\\dev\\tts_regression\\ENU_Ava-ML\\baseline\\Audi_Azure_CVT'
for line in lines:
    o_path=tar_path+'\\'+line.strip('\n')+'.pcm'
    n_path='c:\_workspace\cloud\\rtest\\new_ref'+'\\'+line.strip('\n')+'_ref.pcm'
    shutil.copyfile(o_path,n_path)

'''
bat_file = 'C:\_projects\mib2plus-connected\dev\_tools\cloud\\audi_cvt_azure.bat'
with open(bat_file) as b:
    b_lines = b.readlines()
b.close()
print(b_lines[:5])
'''